<a href="https://colab.research.google.com/github/ibonetc/AI_Deep-Learning/blob/main/EjemploRAGSencillo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.4/31.4 MB 21.7 MB/s eta 0:00:00


In [3]:
from sentence_transformers import SentenceTransformer
import numpy as np
import faiss
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import torch
from textwrap import fill

## Datos

In [4]:
DOCUMENTOS = [
    ("doc_01", "La inmunoterapia es un tratamiento contra el cáncer que estimula el sistema inmunitario del paciente para que ataque las células tumorales."),
    ("doc_02", "Los biomarcadores PD-1 y PD-L1 pueden ayudar a predecir la respuesta a ciertos tratamientos de inmunoterapia en oncología."),
    ("doc_03", "El aprendizaje profundo se usa para clasificar imágenes médicas, detectar tumores y apoyar el diagnóstico temprano de cáncer."),
    ("doc_04", "La quimioterapia utiliza fármacos citotóxicos y puede combinarse con radioterapia o inmunoterapia según el caso clínico."),
    ("doc_05", "Los ensayos clínicos fase III comparan la eficacia y seguridad de tratamientos nuevos frente al estándar de cuidado.")
]

# Embeddings con uno multilingüe pequeño

In [5]:
enc_name = "intfloat/multilingual-e5-small"
encoder = SentenceTransformer(enc_name)

modules.json:   0%|          | 0.00/387 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/57.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/655 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/471M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/443 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/167 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/200 [00:00<?, ?B/s]

## Embedding
En este transformer los documentos deben comenzar con passage

In [6]:
textos = ["passage: " + txt for indexdoc, txt in DOCUMENTOS]
textos

['passage: La inmunoterapia es un tratamiento contra el cáncer que estimula el sistema inmunitario del paciente para que ataque las células tumorales.',
 'passage: Los biomarcadores PD-1 y PD-L1 pueden ayudar a predecir la respuesta a ciertos tratamientos de inmunoterapia en oncología.',
 'passage: El aprendizaje profundo se usa para clasificar imágenes médicas, detectar tumores y apoyar el diagnóstico temprano de cáncer.',
 'passage: La quimioterapia utiliza fármacos citotóxicos y puede combinarse con radioterapia o inmunoterapia según el caso clínico.',
 'passage: Los ensayos clínicos fase III comparan la eficacia y seguridad de tratamientos nuevos frente al estándar de cuidado.']

In [7]:
ids  = [doc_id for doc_id, _ in DOCUMENTOS]
ids

['doc_01', 'doc_02', 'doc_03', 'doc_04', 'doc_05']

In [8]:
vec_search = encoder.encode(textos, batch_size=32, convert_to_numpy=True, normalize_embeddings=True)

## Index FAISS

In [9]:
index = faiss.IndexFlatIP(vec_search.shape[1])  # IP = inner product
index.add(vec_search)

## Generador

In [10]:
gen_name = "google/flan-t5-small"
gen_tok  = AutoTokenizer.from_pretrained(gen_name)
gen_model= AutoModelForSeq2SeqLM.from_pretrained(gen_name)

tokenizer_config.json: 0.00B [00:00, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json: 0.00B [00:00, ?B/s]

config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/308M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

## Recuperación

In [11]:
def retrieve(query, k=3):
  q_vec = encoder.encode([f"query: {query}"], convert_to_numpy=True, normalize_embeddings=True)
  scores, idxs = index.search(q_vec, k)  # scores: [1, k], idxs: [1, k]
  resultados = []
  for score, idx in zip(scores[0], idxs[0]):
      resultados.append((ids[idx], DOCUMENTOS[idx][1], float(score)))
  return resultados

In [12]:
pregunta="Qué es la inmunoterapia?"
top=retrieve(pregunta)
top

[('doc_01',
  'La inmunoterapia es un tratamiento contra el cáncer que estimula el sistema inmunitario del paciente para que ataque las células tumorales.',
  0.9341772794723511),
 ('doc_04',
  'La quimioterapia utiliza fármacos citotóxicos y puede combinarse con radioterapia o inmunoterapia según el caso clínico.',
  0.868862509727478),
 ('doc_02',
  'Los biomarcadores PD-1 y PD-L1 pueden ayudar a predecir la respuesta a ciertos tratamientos de inmunoterapia en oncología.',
  0.8442903757095337)]

In [14]:
context = [t[1] for t in top]
context

['La inmunoterapia es un tratamiento contra el cáncer que estimula el sistema inmunitario del paciente para que ataque las células tumorales.',
 'La quimioterapia utiliza fármacos citotóxicos y puede combinarse con radioterapia o inmunoterapia según el caso clínico.',
 'Los biomarcadores PD-1 y PD-L1 pueden ayudar a predecir la respuesta a ciertos tratamientos de inmunoterapia en oncología.']

In [15]:
contex_text = "\n\n".join([f"- {c}" for c in context])
contex_text

'- La inmunoterapia es un tratamiento contra el cáncer que estimula el sistema inmunitario del paciente para que ataque las células tumorales.\n\n- La quimioterapia utiliza fármacos citotóxicos y puede combinarse con radioterapia o inmunoterapia según el caso clínico.\n\n- Los biomarcadores PD-1 y PD-L1 pueden ayudar a predecir la respuesta a ciertos tratamientos de inmunoterapia en oncología.'

In [16]:
query=pregunta
prompt = (
        "Responde en español de forma concisa usando únicamente el CONTEXTO. "
        "Si falta información, di que no está en el contexto.\n\n"
        f"CONTEXTO:\n{contex_text}\n\n"
        f"PREGUNTA: {query}\n"
        "RESPUESTA:"
    )

In [18]:
inputs = gen_tok(prompt, return_tensors="pt")
inputs

{'input_ids': tensor([[23483,    15,     3,    35, 16159,     9,     2,    32,    40,    20,
          8066,   975,    75,   159,     9,   178,   232,    32,     3,     2,
          2532, 16666,     3,    15,    40,  8472,  3463,     4,  5647,     5,
           925, 12553,    17,     9,  3261,     9, 12765,     6,  1227,   238,
           150,   259,  2975,     3,    35,     3,    15,    40,  2625,    32,
             5,  8472,  3463,     4,  5647,    10,     3,    18,   325,    16,
            51,   202,    32, 29642,     3,    15,     7,    73, 16104,  3690,
            35,   235,  5314,     3,    15,    40,     3,    75, 12916,  2110,
           238, 12182,    83,     9,     3,    15,    40,  6517,     9,    16,
         11760,  5310,    32,    20,    40, 15531,    15,  3856,   238,    44,
             9,   835,    50,     7,     3,    75,   154,    40,    83,     9,
             7,  8985,  4529,     5,     3,    18,   325,   546,  4133,    32,
         29642, 18187,     3,    89,  

In [19]:
with torch.no_grad():
  outputs = gen_model.generate(
      **inputs,
      max_new_tokens=256,
      temperature=0.2,
      do_sample=False
  )

The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


In [20]:
respuesta=gen_tok.decode(outputs[0], skip_special_tokens=True)
respuesta

'- La inmunoterapia es un tratamiento contra el cáncer que estimula el sistema inmunitario del paciente para que ataque las células tumorales. - La quimioterapia utiliza fármacos citotóxicos y puede combinarse con radioterapia o inmunoterapia segn el caso clnico. - Los biomarcadores PD-1 y PD-L1 pueden ayudar a predecir la respuesta aciertos tratamientos de inmunoterapia en oncologa. PREGUNTA: Quién es la inmunoterapia?'

In [15]:
def generar_respuesta(query, context_chunks, max_new_tokens=200):
    contexto = "\n\n".join([f"- {c}" for c in context_chunks])
    prompt = (
        "Responde en español de forma concisa usando únicamente el CONTEXTO. "
        "Si falta información, di que no está en el contexto.\n\n"
        f"CONTEXTO:\n{contexto}\n\n"
        f"PREGUNTA: {query}\n"
        "RESPUESTA:"
    )
    inputs = gen_tok(prompt, return_tensors="pt")
    with torch.no_grad():
      outputs = gen_model.generate(
        **inputs,
        max_new_tokens=max_new_tokens,
        temperature=0.2,
        do_sample=False
      )
    return gen_tok.decode(outputs[0], skip_special_tokens=True)

In [16]:
def rag_qa(pregunta, k=3):
  top = retrieve(pregunta, k=k)
  context = [t[1] for t in top]
  respuesta = generar_respuesta(pregunta, context)
  print("\n" + "="*80)
  print("PREGUNTA:", pregunta)
  print("-"*80)
  print("TOP-k CONTEXTO (id, score):")
  for (doc_id, txt, sc) in top:
      print(f"  {doc_id} (sim={sc:.3f}) -> {fill(txt, 100)}")
  print("-"*80)
  print("RESPUESTA:")
  print(fill(respuesta, 100))
  print("="*80 + "\n")
  return respuesta

In [17]:
rag_qa("¿Qué es la inmunoterapia y para qué se usa?")

The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.



PREGUNTA: ¿Qué es la inmunoterapia y para qué se usa?
--------------------------------------------------------------------------------
TOP-k CONTEXTO (id, score):
  doc_01 (sim=0.926) -> La inmunoterapia es un tratamiento contra el cáncer que estimula el sistema inmunitario del paciente
para que ataque las células tumorales.
  doc_04 (sim=0.872) -> La quimioterapia utiliza fármacos citotóxicos y puede combinarse con radioterapia o inmunoterapia
según el caso clínico.
  doc_02 (sim=0.848) -> Los biomarcadores PD-1 y PD-L1 pueden ayudar a predecir la respuesta a ciertos tratamientos de
inmunoterapia en oncología.
--------------------------------------------------------------------------------
RESPUESTA:
- La inmunoterapia es un tratamiento contra el cáncer que estimula el sistema inmunitario del
paciente para que ataque las células tumorales. - La quimioterapia utiliza fármacos citotóxicos y
puede combinarse con radioterapia o inmunoterapia segn el caso clnico. - Los biomarcadores PD-1 

'- La inmunoterapia es un tratamiento contra el cáncer que estimula el sistema inmunitario del paciente para que ataque las células tumorales. - La quimioterapia utiliza fármacos citotóxicos y puede combinarse con radioterapia o inmunoterapia segn el caso clnico. - Los biomarcadores PD-1 y PD-L1 pueden ayudar a predecir la respuesta aciertos tratamientos de inmunoterapia en oncologa. PREGUNTA: Qué es la inmunoterapia y para qué se us'

In [18]:
rag_qa("¿Cómo ayudan PD-1 o PD-L1 en el tratamiento?")



PREGUNTA: ¿Cómo ayudan PD-1 o PD-L1 en el tratamiento?
--------------------------------------------------------------------------------
TOP-k CONTEXTO (id, score):
  doc_02 (sim=0.903) -> Los biomarcadores PD-1 y PD-L1 pueden ayudar a predecir la respuesta a ciertos tratamientos de
inmunoterapia en oncología.
  doc_01 (sim=0.835) -> La inmunoterapia es un tratamiento contra el cáncer que estimula el sistema inmunitario del paciente
para que ataque las células tumorales.
  doc_04 (sim=0.831) -> La quimioterapia utiliza fármacos citotóxicos y puede combinarse con radioterapia o inmunoterapia
según el caso clínico.
--------------------------------------------------------------------------------
RESPUESTA:
How can biomarcadores PD-1 and PD-L1 help predecir la respuesta aciertos tratamientos de
inmunoterapia en oncologa. - La inmunoterapia es un tratamiento contra el cáncer que estimula el
sistema inmunitario del paciente para que ataque las células tumorales. - La quimioterapia utiliza
fá

'How can biomarcadores PD-1 and PD-L1 help predecir la respuesta aciertos tratamientos de inmunoterapia en oncologa. - La inmunoterapia es un tratamiento contra el cáncer que estimula el sistema inmunitario del paciente para que ataque las células tumorales. - La quimioterapia utiliza fármacos citotóxicos y puede combinarse con radioterapia o inmunoterapia segn el caso clnico.'

In [19]:
rag_qa("¿Se usa aprendizaje profundo en diagnóstico de cáncer?")


PREGUNTA: ¿Se usa aprendizaje profundo en diagnóstico de cáncer?
--------------------------------------------------------------------------------
TOP-k CONTEXTO (id, score):
  doc_03 (sim=0.949) -> El aprendizaje profundo se usa para clasificar imágenes médicas, detectar tumores y apoyar el
diagnóstico temprano de cáncer.
  doc_02 (sim=0.821) -> Los biomarcadores PD-1 y PD-L1 pueden ayudar a predecir la respuesta a ciertos tratamientos de
inmunoterapia en oncología.
  doc_01 (sim=0.811) -> La inmunoterapia es un tratamiento contra el cáncer que estimula el sistema inmunitario del paciente
para que ataque las células tumorales.
--------------------------------------------------------------------------------
RESPUESTA:
El aprendizaje profundo se usa para clasificar imágenes médicas, detectar tumores y apoyar el
diagnóstico temprano de cáncer. - Los biomarcadores PD-1 y PD-L1 pueden ayudar a predecir la
respuesta aciertos tratamientos de inmunoterapia en oncologa. - La inmunoterapia es u

'El aprendizaje profundo se usa para clasificar imágenes médicas, detectar tumores y apoyar el diagnóstico temprano de cáncer. - Los biomarcadores PD-1 y PD-L1 pueden ayudar a predecir la respuesta aciertos tratamientos de inmunoterapia en oncologa. - La inmunoterapia es un tratamiento contra el cáncer que estimula el sistema inmunitario del paciente para que ataque las células tumorales. PREGUNTA: Se usa aprendizaje profundo en diagnósti'